# 1. Import

## 1.1 Import Bib



In [ ]:
import pandas as pd

import geopandas as gpd
from shapely.geometry import LineString, Point

from branca.colormap import StepColormap

import folium
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown

from folium.features import GeoJsonTooltip

from IPython.display import display

from branca.colormap import linear

## 1.2 Import Data

In [ ]:
#csv einlesen
wege = pd.read_csv("generated_data/wege_freizeit_erweitert.csv", sep=";")

In [ ]:
#csv einlesen
etappen = pd.read_csv("generated_data/etappen_freizeit_erweitert.csv", sep=";")

In [ ]:

gemeinden = pd.read_csv("Data/Ortschaftsverzeichnis.csv", sep=";")

In [ ]:
# File path to the Shapefile
shapefile_path = 'Data/swissBOUNDARIES3D_1_5_TLM_KANTONSGEBIET.shp'

# Load the Shapefile
shp_data_kanton = gpd.read_file(shapefile_path)

In [ ]:
# File path to the Shapefile
shapefile_path = 'Data/swissBOUNDARIES3D_1_5_TLM_HOHEITSGEBIET.shp'

# Load the Shapefile
shp_data_gemeinde = gpd.read_file(shapefile_path)

# 2. Data bearbeiten

In [ ]:
wege.head()

In [ ]:
wege.columns

In [ ]:
etappen.head()

In [ ]:
etappen.columns

In [ ]:
gemeinden.head()

# 3. Merge

In [ ]:
# Merge über PLZ: Wohnort PLZ von Wege und PLZ von Gemeinden
wege_gemeinden_merged = wege.merge(
    gemeinden[['PLZ', 'Gemeindename']],  # Nur relevante Spalten für den Merge
    left_on='Wohnort: PLZ',  # Spalte im Wege-Dataset
    right_on='PLZ',  # Spalte im Gemeinden-Dataset
    how='left'  # Alle Wege behalten, auch wenn keine passende Gemeinde gefunden wird
)

# Die Spalte PLZ aus Gemeinden entfernen, falls sie nicht mehr benötigt wird
wege_gemeinden_merged.drop(columns=['PLZ'], inplace=True)

# Ergebnis anzeigen
wege_gemeinden_merged

# 4. Analysen

## 4.1 Wege

In [ ]:
shp_data_kanton.head()

In [ ]:
#Zeig mir die unique values der Spalte NAME
shp_data_kanton['NAME'].unique()

In [ ]:
# Mapping von Kantonsnamen zu Kantonskürzeln
kantons_mapping = {
    'Genève': 'GE', 'Thurgau': 'TG', 'Valais': 'VS', 'Aargau': 'AG', 'Schwyz': 'SZ', 
    'Zürich': 'ZH', 'Obwalden': 'OW', 'Fribourg': 'FR', 'Glarus': 'GL', 'Uri': 'UR',
    'Nidwalden': 'NW', 'Solothurn': 'SO', 'Appenzell Ausserrhoden': 'AR', 'Jura': 'JU', 
    'Graubünden': 'GR', 'Vaud': 'VD', 'Luzern': 'LU', 'Ticino': 'TI', 'Zug': 'ZG', 
    'Basel-Landschaft': 'BL', 'St. Gallen': 'SG', 'Schaffhausen': 'SH', 'Bern': 'BE', 
    'Basel-Stadt': 'BS', 'Neuchâtel': 'NE', 'Appenzell Innerrhoden': 'AI'
}

# Neue Spalte für die Kantonskürzel hinzufügen
shp_data_kanton['KANTONS_KÜRZEL'] = shp_data_kanton['NAME'].map(kantons_mapping)

# Ergebnisse überprüfen
print(shp_data_kanton[['NAME', 'KANTONS_KÜRZEL']].head())


In [ ]:
wege_gemeinden_merged.head()

4.1.1 Gesamtemissionen pro Kanton

In [ ]:
# Sicherstellen, dass alle Daten serialisierbar sind
def prepare_serializable_data(gdf):
    for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime']).columns:
        gdf[col] = gdf[col].astype(str)  # Konvertiere Timestamp in String
    return gdf

# Funktion, um die Karte für alle Freizeitaktivitäten zu plotten
def plot_map_for_all_activities():
    # Median-Emissionen pro Gemeinde berechnen (ohne Filter)
    median_emissionen_gemeinde = wege_gemeinden_merged.groupby("Gemeindename")["Emissionen"].mean().reset_index()
    median_emissionen_gemeinde.rename(columns={"Gemeindename": "NAME", "Emissionen": "Median_Emissionen"}, inplace=True)

    # Shapefile mit aggregierten Daten verbinden
    shp_data_gemeinde_filtered = shp_data_gemeinde.merge(median_emissionen_gemeinde, on="NAME", how="left")

    # Aggregation auf Kantonsebene
    kanton_emissionen = shp_data_gemeinde_filtered.groupby("KANTONSNUM")["Median_Emissionen"].median().reset_index()
    kanton_emissionen.rename(columns={"KANTONSNUM": "KANTONSNUM", "Median_Emissionen": "Median_Emissionen_Kanton"}, inplace=True)

    # Kanton-Shapefile mit aggregierten Daten verbinden
    shp_data_kanton_filtered = shp_data_kanton.merge(kanton_emissionen, on="KANTONSNUM", how="left")

    # Konvertieren in WGS84 (für folium erforderlich)
    shp_data_kanton_filtered = shp_data_kanton_filtered.to_crs(epsg=4326)

    # Sicherstellen, dass die Daten JSON-kompatibel sind
    shp_data_kanton_filtered = prepare_serializable_data(shp_data_kanton_filtered)

    # Median-Emissionen im Schweizer Format formatieren
    shp_data_kanton_filtered['Formatted_Emissionen'] = shp_data_kanton_filtered['Median_Emissionen_Kanton'].apply(
        lambda x: f"{x:,.0f}".replace(",", "'") if not pd.isnull(x) else None
    )

    # Blaue Farbskala erstellen (basierend auf Emissionen)
    max_emission = shp_data_kanton_filtered["Median_Emissionen_Kanton"].max()
    colormap = linear.Blues_09.scale(0, max_emission)  # Blaue Farbskala
    colormap.caption = "Mittlere Emissionen (kg CO2) pro Kanton"

    # Interaktive Karte erstellen (leerer Hintergrund für weiße Karte)
    m = folium.Map(
        location=[46.8, 8.3],  # Zentrale Position (Schweiz)
        zoom_start=8,  # Zoom-Level
        tiles=None  # Entferne die Standard-Hintergrundkachel
    )
    
    # Weißes Hintergrund-Overlay hinzufügen
    folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr='White Background',
        name='White',
        show=False,
        control=False,
        opacity=0  # Unsichtbar machen, für weißen Hintergrund
    ).add_to(m)

    # GeoJson-Layer hinzufügen, um Kantone und Tooltips anzuzeigen
    folium.GeoJson(
        shp_data_kanton_filtered,
        style_function=lambda feature: {
            'fillColor': (
                "lightgrey" if feature['properties']['Median_Emissionen_Kanton'] is None else
                "white" if feature['properties']['Median_Emissionen_Kanton'] == 0 else
                colormap(feature['properties']['Median_Emissionen_Kanton'])
            ),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME', 'Formatted_Emissionen'],  # Zeige Name und formatierte Emissionen
            aliases=['Kantonsname:', 'Mittlere Emissionen (kg CO2):'],  # Überschriften
            localize=True
        ),
    ).add_to(m)

    # Kantonskürzel als fett geschriebenen Text auf der Karte hinzufügen
    for _, row in shp_data_kanton_filtered.iterrows():
        if row['geometry'].centroid:
            folium.Marker(
                location=[row['geometry'].centroid.y, row['geometry'].centroid.x],  # Zentrum des Kantons
                icon=folium.DivIcon(
                    html=f'<div style="font-size: 12px; font-weight: bold; color: black; text-align: center;">{row["KANTONS_KÜRZEL"]}</div>'
                )
            ).add_to(m)

    # Farblegende hinzufügen
    colormap.add_to(m)

    # Rückgabe der Karte
    return m

# Karte direkt anzeigen
m = plot_map_for_all_activities()
display(m)


4.1.2 Gesamtemissionen pro Freizeitkativität und Kanton

In [ ]:
# Sicherstellen, dass alle Daten serialisierbar sind
def prepare_serializable_data(gdf):
    for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime']).columns:
        gdf[col] = gdf[col].astype(str)  # Konvertiere Timestamp in String
    return gdf

# Funktion, um die Karte basierend auf der Freizeitaktivität zu plotten
def plot_map_by_kanton(activity):
    # Daten für die ausgewählte Freizeitaktivität filtern
    filtered_data = wege_gemeinden_merged[wege_gemeinden_merged["Freizeitaktivität"] == activity]
    
    # Durchschnittliche Emissionen pro Gemeinde berechnen
    avg_emissionen_gemeinde = filtered_data.groupby("Gemeindename")["Emissionen"].mean().reset_index()
    avg_emissionen_gemeinde.rename(columns={"Gemeindename": "NAME", "Emissionen": "Avg_Emissionen"}, inplace=True)

    # Shapefile mit gefilterten Daten verbinden
    shp_data_gemeinde_filtered = shp_data_gemeinde.merge(avg_emissionen_gemeinde, on="NAME", how="left")

    # Aggregation auf Kantonsebene
    kanton_emissionen = shp_data_gemeinde_filtered.groupby("KANTONSNUM")["Avg_Emissionen"].mean().reset_index()
    kanton_emissionen.rename(columns={"KANTONSNUM": "KANTONSNUM", "Avg_Emissionen": "Avg_Emissionen_Kanton"}, inplace=True)

    # Kanton-Shapefile mit aggregierten Daten verbinden
    shp_data_kanton_filtered = shp_data_kanton.merge(kanton_emissionen, on="KANTONSNUM", how="left")

    # Konvertieren in WGS84 (für folium erforderlich)
    shp_data_kanton_filtered = shp_data_kanton_filtered.to_crs(epsg=4326)

    # Sicherstellen, dass die Daten JSON-kompatibel sind
    shp_data_kanton_filtered = prepare_serializable_data(shp_data_kanton_filtered)

    # Durchschnittliche Emissionen im Schweizer Format formatieren (mit Runden auf ganze Zahlen)
    shp_data_kanton_filtered['Formatted_Emissionen'] = shp_data_kanton_filtered['Avg_Emissionen_Kanton'].apply(
        lambda x: f"{x:,.0f}".replace(",", "'") if not pd.isnull(x) else None
    )

    # Blaue Farbskala erstellen (basierend auf Emissionen)
    max_emission = shp_data_kanton_filtered["Avg_Emissionen_Kanton"].max()
    colormap = linear.Blues_09.scale(0, max_emission)  # Blaue Farbskala
    colormap.caption = "Durchschnittliche Emissionen (kg CO2) pro Kanton"

    # Interaktive Karte erstellen (weißer Hintergrund)
    m = folium.Map(
        location=[46.8, 8.3],  # Zentrale Position (Schweiz)
        zoom_start=8,  # Zoom-Level
        tiles=None  # Entferne die Standard-Hintergrundkachel
    )

    # Weißes Hintergrund-Overlay hinzufügen
    folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr='White Background',
        name='White Background',
        show=False,
        control=False,
        opacity=0  # Unsichtbar, um weißen Hintergrund zu haben
    ).add_to(m)

    # GeoJson-Layer hinzufügen, um Kantone und Tooltips anzuzeigen
    folium.GeoJson(
        shp_data_kanton_filtered,
        style_function=lambda feature: {
            'fillColor': (
                "lightgrey" if feature['properties']['Avg_Emissionen_Kanton'] is None else
                "white" if feature['properties']['Avg_Emissionen_Kanton'] == 0 else
                colormap(feature['properties']['Avg_Emissionen_Kanton'])
            ),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME', 'Formatted_Emissionen'],  # Zeige Kanton und formatierte Emissionen
            aliases=['Kantonsname:', 'Durchschnittliche Emissionen (kg CO2):'],  # Überschriften
            localize=True
        ),
    ).add_to(m)

    # Kantonskürzel als fett geschriebenen Text auf der Karte hinzufügen
    for _, row in shp_data_kanton_filtered.iterrows():
        if row['geometry'].centroid:
            folium.Marker(
                location=[row['geometry'].centroid.y, row['geometry'].centroid.x],  # Zentrum des Kantons
                icon=folium.DivIcon(
                    html=f'<div style="font-size: 12px; font-weight: bold; color: black; text-align: center;">{row["KANTONS_KÜRZEL"]}</div>'
                )
            ).add_to(m)

    # Farblegende hinzufügen
    colormap.add_to(m)

    # Rückgabe der Karte
    return m

# Dropdown mit allen Freizeitaktivitäten erstellen
freizeitaktivitaeten = wege_gemeinden_merged["Freizeitaktivität"].unique()

# Interact korrekt einrichten
def display_map(activity):
    m = plot_map_by_kanton(activity)  # Ruft die Kartenfunktion auf
    display(m)

interact(display_map, activity=Dropdown(options=freizeitaktivitaeten, description="Freizeitaktivität:"))


4.1.3 Gesamtemissionen pro Gemeinde

In [ ]:
from branca.colormap import StepColormap
import folium
from folium import GeoJsonTooltip
import pandas as pd

# Sicherstellen, dass alle Daten serialisierbar sind
def prepare_serializable_data(gdf):
    for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime']).columns:
        gdf[col] = gdf[col].astype(str)  # Konvertiere Timestamp in String
    return gdf

# Funktion, um die Karte für alle Freizeitaktivitäten zu plotten
def plot_map_for_all_activities():
    # Median-Emissionen pro Gemeinde berechnen (ohne Filter)
    median_emissionen = wege_gemeinden_merged.groupby("Gemeindename")["Emissionen"].mean().reset_index()
    median_emissionen.rename(columns={"Gemeindename": "NAME", "Emissionen": "Median_Emissionen"}, inplace=True)

    # Shapefile mit aggregierten Daten verbinden
    shp_data_filtered = shp_data_gemeinde.merge(median_emissionen, on="NAME", how="left")

    # Konvertieren in WGS84 (für folium erforderlich)
    shp_data_filtered = shp_data_filtered.to_crs(epsg=4326)

    # Sicherstellen, dass die Daten JSON-kompatibel sind
    shp_data_filtered = prepare_serializable_data(shp_data_filtered)

    # Median-Emissionen im Schweizer Format formatieren (mit Runden auf ganze Zahlen)
    shp_data_filtered['Formatted_Emissionen'] = shp_data_filtered['Median_Emissionen'].apply(
        lambda x: f"{x:,.0f}".replace(",", "'") if not pd.isnull(x) else None
    )

    # Blaue Farbskala erstellen (dunkelblau ab 20'000)
    cutoff_value = 20000  # Schwellenwert für Dunkelblau
    colormap = StepColormap(
        colors=["#f7fbff", "#c6dbef", "#6baed6", "#2171b5", "#08306b"],  # Blautöne
        index=[0, 5000, 10000, 15000, cutoff_value, shp_data_filtered["Median_Emissionen"].max()],
        vmin=0,
        vmax=shp_data_filtered["Median_Emissionen"].max(),
        caption="Mittlere Emissionen (kg CO2)"
    )

    # Interaktive Karte erstellen (weißer Hintergrund)
    m = folium.Map(
        location=[46.8, 8.3],  # Zentrale Position (Schweiz)
        zoom_start=8,  # Zoom-Level
        tiles=None  # Entferne die Standard-Hintergrundkachel
    )

    # Weißes Hintergrund-Overlay hinzufügen
    folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr='White Background',
        name='White Background',
        show=False,
        control=False,
        opacity=0  # Unsichtbar, um einen weißen Hintergrund zu erzeugen
    ).add_to(m)

    # GeoJson-Layer hinzufügen, um Gemeinden und Tooltips anzuzeigen
    folium.GeoJson(
        shp_data_filtered,
        style_function=lambda feature: {
            'fillColor': (
                "lightgrey" if feature['properties']['Median_Emissionen'] is None else
                "white" if feature['properties']['Median_Emissionen'] == 0 else
                colormap(
                    min(feature['properties']['Median_Emissionen'], cutoff_value)  # Werte über 20'000 begrenzen
                )
            ),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME', 'Formatted_Emissionen'],  # Zeige Name und formatierte Emissionen
            aliases=['Gemeindename:', 'Mittlere Emissionen (kg CO2):'],  # Überschriften
            localize=True
        ),
    ).add_to(m)

    # Farblegende hinzufügen
    colormap.add_to(m)

    # Rückgabe der Karte
    return m

# Karte direkt anzeigen
m = plot_map_for_all_activities()
display(m)


4.1.4 Gesamtemissionen pro Freizeitaktivität und Gemeinde

In [44]:
# Sicherstellen, dass alle Daten serialisierbar sind
def prepare_serializable_data(gdf):
    for col in gdf.select_dtypes(include=['datetime64[ns]', 'datetime']).columns:
        gdf[col] = gdf[col].astype(str)  # Konvertiere Timestamp in String
    return gdf

# Funktion, um die Karte basierend auf der Freizeitaktivität zu plotten
def plot_map(activity):
    # Daten für die ausgewählte Freizeitaktivität filtern
    filtered_data = wege_gemeinden_merged[wege_gemeinden_merged["Freizeitaktivität"] == activity]
    
    # Durchschnittliche Emissionen pro Gemeinde berechnen
    avg_emissionen = filtered_data.groupby("Gemeindename")["Emissionen"].mean().reset_index()
    avg_emissionen.rename(columns={"Gemeindename": "NAME", "Emissionen": "Avg_Emissionen"}, inplace=True)

    # Shapefile mit gefilterten Daten verbinden
    shp_data_filtered = shp_data_gemeinde.merge(avg_emissionen, on="NAME", how="left")

    # Konvertieren in WGS84 (für folium erforderlich)
    shp_data_filtered = shp_data_filtered.to_crs(epsg=4326)

    # Sicherstellen, dass die Daten JSON-kompatibel sind
    shp_data_filtered = prepare_serializable_data(shp_data_filtered)

    # Durchschnittliche Emissionen im Schweizer Format formatieren (mit Runden auf ganze Zahlen)
    shp_data_filtered['Formatted_Emissionen'] = shp_data_filtered['Avg_Emissionen'].apply(
        lambda x: f"{x:,.0f}".replace(",", "'") if not pd.isnull(x) else None
    )

    # Blaue Farbskala erstellen (dunkelblau ab 20'000)
    cutoff_value = 20000  # Schwellenwert für Dunkelblau
    colormap = StepColormap(
        colors=["#f7fbff", "#c6dbef", "#6baed6", "#2171b5", "#08306b"],  # Blautöne
        index=[0, 5000, 10000, 15000, cutoff_value, shp_data_filtered["Avg_Emissionen"].max()],
        vmin=0,
        vmax=shp_data_filtered["Avg_Emissionen"].max(),
        caption="Durchschnittliche Emissionen (kg CO2)"
    )

    # Interaktive Karte erstellen (weißer Hintergrund)
    m = folium.Map(
        location=[46.8, 8.3],  # Zentrale Position (Schweiz)
        zoom_start=8,  # Zoom-Level
        tiles=None  # Entferne die Standard-Hintergrundkachel
    )

    # Weißes Hintergrund-Overlay hinzufügen
    folium.TileLayer(
        tiles='https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr='White Background',
        name='White Background',
        show=False,
        control=False,
        opacity=0  # Unsichtbar, um den Hintergrund weiß zu machen
    ).add_to(m)

    # GeoJson-Layer hinzufügen, um Gemeinden und Tooltips anzuzeigen
    folium.GeoJson(
        shp_data_filtered,
        style_function=lambda feature: {
            'fillColor': (
                "lightgrey" if feature['properties']['Avg_Emissionen'] is None else
                "white" if feature['properties']['Avg_Emissionen'] == 0 else
                colormap(
                    min(feature['properties']['Avg_Emissionen'], cutoff_value)  # Werte über 20'000 begrenzen
                )
            ),
            'color': 'black',
            'weight': 0.5,
            'fillOpacity': 0.7,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME', 'Formatted_Emissionen'],  # Zeige Name und formatierte Emissionen
            aliases=['Gemeindename:', 'Durchschnittliche Emissionen (kg CO2):'],  # Überschriften
            localize=True
        ),
    ).add_to(m)

    # Farblegende hinzufügen
    colormap.add_to(m)

    # Rückgabe der Karte
    return m

# Dropdown mit allen Freizeitaktivitäten erstellen
freizeitaktivitaeten = wege_gemeinden_merged["Freizeitaktivität"].unique()

# Interact korrekt einrichten
def display_map(activity):
    m = plot_map(activity)  # Ruft die Kartenfunktion auf
    display(m)

interact(display_map, activity=Dropdown(options=freizeitaktivitaeten, description="Freizeitaktivität:"))


In [ ]:
# Überprüfen, ob wichtige Spalten keine fehlenden Werte enthalten
required_columns = [
    'Start_X-Koordinate', 'Start_Y-Koordinate', 
    'Ziel_X-Koordinate', 'Ziel_Y-Koordinate', 
    'Freizeitaktivität', 'Emissionen'
]
data_cleaned = wege.dropna(subset=required_columns)
# Geometrien erstellen: Start- und Zielpunkte sowie Linien
data_cleaned['Start_Point'] = data_cleaned.apply(
    lambda row: Point(row['Start_X-Koordinate'], row['Start_Y-Koordinate']), axis=1)
data_cleaned['End_Point'] = data_cleaned.apply(
    lambda row: Point(row['Ziel_X-Koordinate'], row['Ziel_Y-Koordinate']), axis=1)
data_cleaned['Route'] = data_cleaned.apply(
    lambda row: LineString([row['Start_Point'], row['End_Point']]), axis=1)

# In ein GeoDataFrame umwandeln
gdf = gpd.GeoDataFrame(data_cleaned, geometry='Route', crs="EPSG:4326")


In [ ]:
# Karte erstellen (Zentriert auf die Schweiz)
map_emissionen = folium.Map(location=[47, 8], zoom_start=8)

# Freizeitaktivitäten abrufen
freizeitaktivitaeten = gdf['Freizeitaktivität'].unique()

# Layer für jede Freizeitaktivität erstellen
for activity in freizeitaktivitaeten:
    # FeatureGroup für die Freizeitaktivität erstellen
    feature_group = folium.FeatureGroup(name=activity)
    
    # Subset der Daten für die aktuelle Freizeitaktivität
    subset = gdf[gdf['Freizeitaktivität'] == activity]
    
    # Wege für die aktuelle Freizeitaktivität hinzufügen
    for _, row in subset.iterrows():
        emission_color = 'green' if row['Emissionen'] == 0 else 'red'  # Farben basierend auf Emissionen
        folium.PolyLine(
            locations=[
                [row['Start_Y-Koordinate'], row['Start_X-Koordinate']],
                [row['Ziel_Y-Koordinate'], row['Ziel_X-Koordinate']]
            ],
            color=emission_color,
            weight=5,
            tooltip=(
                f"Freizeitaktivität: {row['Freizeitaktivität']}<br>"
                f"Emissionen: {row['Emissionen']} g CO2<br>"
                f"Dauer: {row['Dauer in Minuten']} Minuten"
            )
        ).add_to(feature_group)  # Hier wird die Linie nur zum spezifischen Layer hinzugefügt
    
    # FeatureGroup zur Karte hinzufügen
    feature_group.add_to(map_emissionen)

# LayerControl hinzufügen, um zwischen den Freizeitaktivitäten umzuschalten
folium.LayerControl().add_to(map_emissionen)

# Karte speichern
map_emissionen.save("emissionen_karte.html")